In [1]:
#!pip install fastembed
#!pip install -q "qdrant-client[fastembed]>=1.14.2"

In [2]:
from fastembed import TextEmbedding

/Users/vpankrat/Courses/LLMzoomcamp/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from qdrant_client import QdrantClient, models

client = QdrantClient("http://localhost:6333")
client.get_collections()

CollectionsResponse(collections=[])

# Q1. Embedding the query

In [4]:
import json

EMBEDDING_DIMENSIONALITY = 512

for model in TextEmbedding.list_supported_models():
    if model["dim"] == EMBEDDING_DIMENSIONALITY:
        print(json.dumps(model, indent=2))

{
  "model": "BAAI/bge-small-zh-v1.5",
  "sources": {
    "hf": "Qdrant/bge-small-zh-v1.5",
    "url": "https://storage.googleapis.com/qdrant-fastembed/fast-bge-small-zh-v1.5.tar.gz",
    "_deprecated_tar_struct": true
  },
  "model_file": "model_optimized.onnx",
  "description": "Text embeddings, Unimodal (text), Chinese, 512 input tokens truncation, Prefixes for queries/documents: not so necessary, 2023 year.",
  "license": "mit",
  "size_in_GB": 0.09,
  "additional_files": [],
  "dim": 512,
  "tasks": {}
}
{
  "model": "Qdrant/clip-ViT-B-32-text",
  "sources": {
    "hf": "Qdrant/clip-ViT-B-32-text",
    "url": null,
    "_deprecated_tar_struct": false
  },
  "model_file": "model.onnx",
  "description": "Text embeddings, Multimodal (text&image), English, 77 input tokens truncation, Prefixes for queries/documents: not necessary, 2021 year",
  "license": "mit",
  "size_in_GB": 0.25,
  "additional_files": [],
  "dim": 512,
  "tasks": {}
}
{
  "model": "jinaai/jina-embeddings-v2-small-e

In [5]:
model_handle = 'jinaai/jina-embeddings-v2-small-en'

In [6]:
query = 'I just discovered the course. Can I join now?'

In [7]:
embedding_model = TextEmbedding(model_name=model_handle)

In [8]:
embeddings_list = list(embedding_model.embed([query]))
q = embeddings_list[0]

In [9]:
min(q)

np.float64(-0.11726373551188797)

**Q1 answer: -0.11**

In [10]:
import numpy as np
np.linalg.norm(q)

np.float64(1.0)

In [11]:
q.dot(q)

np.float64(1.0)

# Q2. Cosine similarity with another vector

In [12]:
doc = 'Can I still join the course after the start date?'

In [13]:
embeddings_list_2 = list(embedding_model.embed([doc]))
q2 = embeddings_list_2[0]

In [14]:
q.dot(q2)

np.float64(0.9008528856818037)

**Q2 answer: 0.9**

# Q3. Ranking by cosine

In [15]:
documents = [{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
  'section': 'General course-related questions',
  'question': 'Course - When will the course start?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (installed with Anaconda)\nTerraform\nGit\nLook over the prerequisites and syllabus to see if you are comfortable with these subjects.',
  'section': 'General course-related questions',
  'question': 'Course - What can I do before the course starts?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Star the repo! Share it with friends if you find it useful ❣️\nCreate a PR if you see you can improve the text or the structure of the repository.',
  'section': 'General course-related questions',
  'question': 'How can we contribute to the course?',
  'course': 'data-engineering-zoomcamp'}]

In [16]:
V = np.array(list(embedding_model.embed([doc['text'] for doc in documents])))

In [17]:
len(V)

5

In [18]:
display(V.dot(q))
display(V.dot(q2))

array([0.76296845, 0.81823784, 0.80853973, 0.71330785, 0.73044995])

array([0.8106057 , 0.84989066, 0.7991335 , 0.68130468, 0.67392881])

In [19]:
display(np.argmax(V.dot(q)))
display(np.argmax(V.dot(q2)))

np.int64(1)

np.int64(1)

**Q3 answer: 1**

# Q4. Ranking by cosine, version two

In [20]:
V2 = np.array(list(embedding_model.embed([doc['question'] + ' ' + doc['text'] for doc in documents]))) 

In [21]:
display(V2.dot(q))
display(V2.dot(q2))

array([0.8514543 , 0.84365941, 0.84082872, 0.77551577, 0.80860079])

array([0.92732519, 0.85948473, 0.83966023, 0.76320296, 0.76044491])

In [22]:
display(np.argmax(V2.dot(q)))
display(np.argmax(V2.dot(q2)))

np.int64(0)

np.int64(0)

**Q4 answer: 0, because the question in the document is really close in meaning to the query**

# Q5. Selecting the embedding model

In [23]:
min([model["dim"] for model in TextEmbedding.list_supported_models()])

384

**Q5 answer: 384**

# Q6. Indexing with qdrant (2 points)

In [24]:
import requests 

docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()


documents = []

for course in documents_raw:
    course_name = course['course']
    if course_name != 'machine-learning-zoomcamp':
        continue

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [25]:
documents[0]

{'text': 'Machine Learning Zoomcamp FAQ\nThe purpose of this document is to capture frequently asked technical questions.\nWe did this for our data engineering course and it worked quite well. Check this document for inspiration on how to structure your questions and answers:\nData Engineering Zoomcamp FAQ\nIn the course GitHub repository there’s a link. Here it is: https://airtable.com/shryxwLd0COOEaqXo\nwork',
 'section': 'General course-related questions',
 'question': 'How do I sign up?',
 'course': 'machine-learning-zoomcamp'}

In [26]:
EMBEDDING_DIMENSIONALITY_2 = 384
model_handle_2 = 'BAAI/bge-small-en'

In [27]:
# Define the collection name
collection_name = "zoomcamp-hw2"

# Create the collection with specified vector parameters
client.create_collection(
    collection_name=collection_name,
    vectors_config=models.VectorParams(
        size=EMBEDDING_DIMENSIONALITY_2,  # Dimensionality of the vectors
        distance=models.Distance.COSINE  # Distance metric for similarity search
    )
)

True

In [28]:
points = []
id = 0

for course in documents_raw:
    for doc in course['documents']:

        point = models.PointStruct(
            id=id,
            vector=models.Document(text=doc['question'] + ' ' + doc['text'], model=model_handle_2), #embed text locally with "jinaai/jina-embeddings-v2-small-en" from FastEmbed
            payload={
                "text": doc['text'],
                "question": doc['question'],
                "section": doc['section'],
                "course": course['course']
            } #save all needed metadata fields
        )
        points.append(point)

        id += 1

In [29]:
points[0].vector

Document(text="Course - When will the course start? The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.", model='BAAI/bge-small-en', options=None)

In [30]:
client.upsert(
    collection_name=collection_name,
    points=points
)

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [31]:
query = 'I just discovered the course. Can I join now?'

In [32]:
def search(query, limit=1):

    results = client.query_points(
        collection_name=collection_name,
        query=models.Document( #embed the query text locally with "jinaai/jina-embeddings-v2-small-en"
            text=query,
            model=model_handle_2
        ),
        limit=10, # top closest matches
        with_payload=True #to get metadata in the results
    )

    return results

In [33]:
result = search(query)
result.points[0].payload

{'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.',
 'question': 'The course has already started. Can I still join it?',
 'section': 'General course-related questions',
 'course': 'machine-learning-zoomcamp'}

In [34]:
result.points[0].score

0.8703172

**Q6 answer: 0.87**